In [87]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

df= pd.read_csv('results.csv')

df['date'] = pd.to_datetime(df['date'], dayfirst=True)

df = df[df['date'] > '1997-01-01']

def determine_winner_and_loser(row):
    if row['home_score'] > row['away_score']:
        winner = row['home_team']
        loser = row['away_team']
    elif row['home_score'] < row['away_score']:
        winner = row['away_team']
        loser = row['home_team']
    else:
        winner = 'Draw'
        loser = 'Draw'
    return winner, loser

# Apply the function to create new 'winner' and 'loser' columns
df[['winner', 'loser']] = df.apply(determine_winner_and_loser, axis=1).apply(pd.Series)
df.head()

,date,home_team,away_team,home_score,away_score,competition,stadium,city,country,neutral,world_cup,winner,loser
1350,1997-01-04,Ireland,Italy,29,37,NaN,Lansdowne Road,Dublin,Ireland,False,False,Italy,Ireland
1351,1997-01-18,Scotland,Wales,19,34,1997 Five Nations Championship,Murrayfield Stadium,Edinburgh,Scotland,False,False,Wales,Scotland
1352,1997-01-18,Ireland,France,15,32,1997 Five Nations Championship,Lansdowne Road,Dublin,Ireland,False,False,France,Ireland
1353,1997-02-01,England,Scotland,41,13,1997 Five Nations Championship,Twickenham Stadium,London,England,False,False,England,Scotland
1354,1997-02-01,Wales,Ireland,25,26,1997 Five Nations Championship,National Stadium,Cardiff,Wales,False,False,Ireland,Wales


In [88]:
df['ranking_points_home'] = 0
df['ranking_points_away'] = 0
df['margin'] = df['home_score'] - df['away_score']
df['result'] = df['margin'].apply(lambda x: 'home_win' if x > 0 else ('away_win' if x < 0 else 'draw'))

ranking_points = {'Scotland': 80, 'England': 80, 'Wales': 80, 'Italy': 80, 'France': 80, 'Ireland': 80, 'New Zealand': 80, 'Argentina': 80, 'South Africa': 80, 'Australia': 80 }

for i, row in df.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    


    # Update ranking_points_home and ranking_points_away
    df.at[i, 'ranking_points_home'] = ranking_points[home_team]
    df.at[i, 'ranking_points_away'] = ranking_points[away_team]
    
    home_points = ranking_points[home_team] + 3
    away_points = ranking_points[away_team]
    gap = home_points - away_points
    if gap < -10:
        gap = -10
    if gap > 10:
        gap = 10
    if row['winner'] == 'Draw':
        core = gap*0.1
    elif row['winner'] == home_team:
        core = 1 - (gap*0.1)
    else:
        core = 1 + (gap*0.1)
        
    if np.abs(row['home_score'] - row['away_score']) > 15:
        core *= 1.5
        
    if row['world_cup'] == True:
        core *= 2
        
    if row['winner'] != 'Draw':
        ranking_points[row['winner']] += core
        ranking_points[row['loser']] -= core
        
    
df.reset_index(drop=True, inplace=True)
df.head()


,date,home_team,away_team,home_score,away_score,competition,stadium,city,country,neutral,world_cup,winner,loser,ranking_points_home,ranking_points_away,margin,result
0,1997-01-04,Ireland,Italy,29,37,NaN,Lansdowne Road,Dublin,Ireland,False,False,Italy,Ireland,80.0,80.000,-8,away_win
1,1997-01-18,Scotland,Wales,19,34,1997 Five Nations Championship,Murrayfield Stadium,Edinburgh,Scotland,False,False,Wales,Scotland,80.0,80.000,-15,away_win
2,1997-01-18,Ireland,France,15,32,1997 Five Nations Championship,Lansdowne Road,Dublin,Ireland,False,False,France,Ireland,78.7,80.000,-17,away_win
3,1997-02-01,England,Scotland,41,13,1997 Five Nations Championship,Twickenham Stadium,London,England,False,False,England,Scotland,80.0,78.700,28,home_win
4,1997-02-01,Wales,Ireland,25,26,1997 Five Nations Championship,National Stadium,Cardiff,Wales,False,False,Ireland,Wales,81.3,76.945,-1,away_win


In [92]:
def calculate_team_form(df, team_name, current_row_index):
    team_form = []
    count = 0
    games = df.copy()
    games_b = games.iloc[:current_row_index]
    
    for index in range(len(games_b)-1, -1, -1):
        row1 = games_b.iloc[index]

        if row1['home_team'] == team_name or row1['away_team'] == team_name:

            if row1['winner'] == 'draw':
                team_form.append(0.5)
            elif row1['winner'] == team_name:
                team_form.append(1)
            else:
                team_form.append(0)


       
    
    
    return sum(team_form[:5])

# Example usage:
for idx, row in df.iterrows():
    if idx % 99 == 0:
        print('Batch of 100 done')
    df.at[idx, 'home_form'] = calculate_team_form(df, row['home_team'], idx)
    df.at[idx, 'away_form'] = calculate_team_form(df, row['away_team'], idx)

df.head()

Batch of 100 done
Batch of 100 done
Batch of 100 done
Batch of 100 done
Batch of 100 done
Batch of 100 done
Batch of 100 done
Batch of 100 done
Batch of 100 done
Batch of 100 done
Batch of 100 done
Batch of 100 done
Batch of 100 done


,date,home_team,away_team,home_score,away_score,competition,stadium,city,country,neutral,world_cup,winner,loser,ranking_points_home,ranking_points_away,margin,result,home_form,away_form
0,1997-01-04,Ireland,Italy,29,37,NaN,Lansdowne Road,Dublin,Ireland,False,False,Italy,Ireland,80.0,80.000,-8,away_win,0.0,0.0
1,1997-01-18,Scotland,Wales,19,34,1997 Five Nations Championship,Murrayfield Stadium,Edinburgh,Scotland,False,False,Wales,Scotland,80.0,80.000,-15,away_win,0.0,0.0
2,1997-01-18,Ireland,France,15,32,1997 Five Nations Championship,Lansdowne Road,Dublin,Ireland,False,False,France,Ireland,78.7,80.000,-17,away_win,0.0,0.0
3,1997-02-01,England,Scotland,41,13,1997 Five Nations Championship,Twickenham Stadium,London,England,False,False,England,Scotland,80.0,78.700,28,home_win,0.0,0.0
4,1997-02-01,Wales,Ireland,25,26,1997 Five Nations Championship,National Stadium,Cardiff,Wales,False,False,Ireland,Wales,81.3,76.945,-1,away_win,1.0,0.0


In [93]:
df.iloc[-20:-1]

,date,home_team,away_team,home_score,away_score,competition,stadium,city,country,neutral,world_cup,winner,loser,ranking_points_home,ranking_points_away,margin,result,home_form,away_form
1254,2023-03-18,Scotland,Italy,26,14,2023 Six Nations Championship,Murrayfield Stadium,Edinburgh,Scotland,False,False,Scotland,Italy,78.140342,67.716982,12,home_win,2.0,0.0
1255,2023-03-18,France,Wales,41,28,2023 Six Nations Championship,Stade de France,Saint-Denis,France,False,False,France,Wales,86.641598,75.218705,13,home_win,4.0,1.0
1256,2023-03-18,Ireland,England,29,16,2023 Six Nations Championship,Aviva Stadium,Dublin,Ireland,False,False,Ireland,England,88.551057,78.154582,13,home_win,5.0,2.0
1257,2023-07-08,South Africa,Australia,43,12,2023 Rugby Championship,Loftus Versfeld Stadium,Pretoria,South Africa,False,False,South Africa,Australia,85.078682,77.655916,31,home_win,3.0,2.0
1258,2023-07-08,Argentina,New Zealand,12,41,2023 Rugby Championship,Estadio Malvinas Argentinas,Mendoza,Argentina,False,False,New Zealand,Argentina,77.696089,85.146046,-29,away_win,1.0,4.0
1259,2023-07-15,New Zealand,South Africa,35,20,2023 Rugby Championship,Mount Smart Stadium,Auckland,New Zealand,False,False,New Zealand,South Africa,85.978553,85.078682,15,home_win,4.0,3.0
1260,2023-07-15,Australia,Argentina,31,34,2023 Rugby Championship,CommBank Stadium,Sydney,Australia,False,False,Argentina,Australia,77.655916,76.863583,-3,away_win,1.0,1.0
1261,2023-07-29,Australia,New Zealand,7,38,2023 Rugby Championship,Melbourne Cricket Ground,Melbourne,Australia,False,False,New Zealand,Australia,76.276682,86.588566,-31,away_win,1.0,4.0
1262,2023-07-29,South Africa,Argentina,22,21,2023 Rugby Championship,Ellis Park,Johannesburg,South Africa,False,False,South Africa,Argentina,84.468669,78.242816,1,home_win,3.0,2.0
1263,2023-08-05,New Zealand,Australia,23,20,2023 Rugby World Cup warm-up match,Forsyth Barr Stadium,Dunedin,New Zealand,False,False,New Zealand,Australia,86.991783,75.873465,3,home_win,4.0,1.0


## Predicting Margin of Victory/Loss

In [94]:
data = df.copy()
#data = pd.get_dummies(data, columns=['stadium'], prefix='stadium', dtype=int)
data = data[data['date'] > '2000-01-01']
data['neutral'] = data['neutral'].astype(int)
data['world_cup'] = data['world_cup'].astype(int)

train_data = data[data['date'] < '2017-01-01']
test_data = data[data['date'] >= '2017-01-01']
X_train = train_data[['neutral', 'world_cup', 'ranking_points_home', 'ranking_points_away', 'home_form', 'away_form']]
y_train = train_data['margin']
X_test = test_data[['neutral', 'world_cup', 'ranking_points_home', 'ranking_points_away', 'home_form', 'away_form']]
y_test = test_data['margin']

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and test data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [95]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

models = {
    'Linear Regression': LinearRegression(),
    'Random Forest Regressor': RandomForestRegressor(),
    'Support Vector Regressor': SVR()
}

for model_name, model in models.items():
    # Fit the model to the training data
    model.fit(X_train_scaled, y_train)
    
    # Predict on the test data
    y_pred = model.predict(X_test_scaled)
    
    # Calculate mean squared error (MSE) and R-squared (R2) for evaluation
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Print results
    print(f"Model: {model_name}")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"R-squared (R2): {r2:.2f}")
    print("="*50)

Model: Linear Regression
Mean Squared Error (MSE): 201.89
R-squared (R2): 0.45
Model: Random Forest Regressor
Mean Squared Error (MSE): 218.03
R-squared (R2): 0.41
Model: Support Vector Regressor
Mean Squared Error (MSE): 239.27
R-squared (R2): 0.35


## Predicting Match Winner

In [96]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

y_train = train_data['result']
y_test = test_data['result']


clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_scaled, y_train)

# Predict on the test data
y_pred = clf.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# Print results
print('Random Forest Results')
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_rep)

from sklearn.linear_model import LogisticRegression

# Initialize and fit a Logistic Regression classifier
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train_scaled, y_train)

# Predict on the test data
y_pred_logreg = logreg.predict(X_test_scaled)

# Evaluate the Logistic Regression model
accuracy_logreg = accuracy_score(y_test, y_pred_logreg)
classification_rep_logreg = classification_report(y_test, y_pred_logreg)

# Print results for Logistic Regression
print("Logistic Regression Results:")
print(f"Accuracy: {accuracy_logreg:.2f}")
print("Classification Report:")
print(classification_rep_logreg)
print("="*50)

# Initialize and fit another classifier of your choice (e.g., Decision Tree)
from sklearn.tree import DecisionTreeClassifier

# Initialize and fit a Decision Tree classifier
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train_scaled, y_train)

# Predict on the test data
y_pred_decision_tree = decision_tree.predict(X_test_scaled)

# Evaluate the Decision Tree model
accuracy_decision_tree = accuracy_score(y_test, y_pred_decision_tree)
classification_rep_decision_tree = classification_report(y_test, y_pred_decision_tree)

# Print results for Decision Tree
print("Decision Tree Results:")
print(f"Accuracy: {accuracy_decision_tree:.2f}")
print("Classification Report:")
print(classification_rep_decision_tree)
print("="*50)


Random Forest Results
Accuracy: 0.67
Classification Report:
              precision    recall  f1-score   support

    away_win       0.61      0.55      0.58       121
        draw       0.00      0.00      0.00         9
    home_win       0.70      0.78      0.74       185

    accuracy                           0.67       315
   macro avg       0.44      0.44      0.44       315
weighted avg       0.65      0.67      0.66       315

Logistic Regression Results:
Accuracy: 0.73
Classification Report:
              precision    recall  f1-score   support

    away_win       0.71      0.64      0.67       121
        draw       0.00      0.00      0.00         9
    home_win       0.74      0.83      0.79       185

    accuracy                           0.73       315
   macro avg       0.49      0.49      0.49       315
weighted avg       0.71      0.73      0.72       315

Decision Tree Results:
Accuracy: 0.57
Classification Report:
              precision    recall  f1-score   supp

/Users/jamiewelsh/opt/anaconda3/envs/diss/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jamiewelsh/opt/anaconda3/envs/diss/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jamiewelsh/opt/anaconda3/envs/diss/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier